In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Sequential
from keras.optimizers import Adam

In [0]:
img_rows = 28
img_cols = 28
channels = 1

# Input shape
img_shape = (img_rows, img_cols, channels)

# Noise Vector
z_dim = 100

#### Generator

In [0]:
def build_generator(z_dim):
  model = Sequential()
  
  # reshape to 7x7x256 using a dense layer
  model.add(Dense(256 * 7 * 7, input_dim=z_dim))
  model.add(Reshape((7, 7, 256)))
  
  # transposed conv operation 7x7x256 to 14x14x128
  model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
  
  # Batch normalization
  model.add(BatchNormalization())
  
  # LeakyRelu activation
  model.add(LeakyReLU(alpha=0.01))
  
  # transposed conv, 14x14x128 ==> 14x14x64
  model.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding='same'))
  
  # Batch normalization
  model.add(BatchNormalization())
  
  # LeakyRelu activation
  model.add(LeakyReLU(alpha=0.01))
  
  # transposed conv, 14x14x128 ==> 14x14x64
  model.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding='same'))
  
  # tanh activation
  model.add(Activation('tanh'))
  
  return model

#### Discriminator

In [0]:
def build_discriminator(img_shape):
  model = Sequential()
  
  # Conv layer 28x28x1 ==> 14x14x32
  model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))  
  
  # LeakyRelu activation
  model.add(LeakyReLU())
  
  # Conv layer 14x14x32 ==> 7x7x64, batchnorm and Lrelu activation
  model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))
  
  # Conv layer 7x7x64 ==> 3x3x128
  model.add(Conv2D(128, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))
  
  # Output layer with sigmoid activation
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  
  return model

#### Combine gen and dis

In [0]:
def build_gan(gen, dis):
  model = Sequential()
  
  model.add(gen)
  model.add(dis)
  
  return model

In [0]:
# Build and compile the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Build Generator
generator = build_generator(z_dim)

# Keep Discriminator parameters constant for Generator training
discriminator.trainable = False

# Build and compile GAN model with fixed Discriminator to train the Generator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

#### Training

In [0]:
losses = []
accuracies = []
iteration_checkpoints = []


def train(iterations, batch_size, sample_interval):
